In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
!unzip /content/drive/MyDrive/majorproject/data-20240317T092757Z-001.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/valid/Mild_Demented/mild_875.jpg  
  inflating: data/valid/Mild_Demented/mild_867.jpg  
  inflating: data/valid/Mild_Demented/mild_385.jpg  
  inflating: data/valid/Mild_Demented/mild_364.jpg  
  inflating: data/valid/Mild_Demented/mild_45.jpg  
  inflating: data/valid/Mild_Demented/mild_835.jpg  
  inflating: data/valid/Mild_Demented/mild_294.jpg  
  inflating: data/valid/Mild_Demented/mild_103.jpg  
  inflating: data/valid/Mild_Demented/mild_23.jpg  
  inflating: data/valid/Mild_Demented/mild_15.jpg  
  inflating: data/valid/Mild_Demented/mild_688.jpg  
  inflating: data/valid/Mild_Demented/mild_524.jpg  
  inflating: data/valid/Mild_Demented/mild_441.jpg  
  inflating: data/valid/Mild_Demented/mild_599.jpg  
  inflating: data/valid/Mild_Demented/mild_504.jpg  
  inflating: data/valid/Mild_Demented/mild_888.jpg  
  inflating: data/valid/Mild_Demented/mild_890.jpg  
  inflating: data/valid/Mild_Demented/mild_676.jpg 

In [3]:
import os
from collections import Counter
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

# Define the paths to your data folders
train_path = '/content/data/train'
valid_path = '/content/data/valid'
test_path = '/content/data/test'

# Define the image dimensions and batch size
img_width, img_height = 64,64
batch_size = 32

# Data augmentation parameters
data_gen_args = dict(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create ImageDataGenerator for data augmentation and preprocessing
train_datagen = ImageDataGenerator(**data_gen_args, rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the train, valid, and test data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='sparse',
    shuffle=False
)

# Get the class labels and number of classes
class_labels = train_generator.class_indices
num_classes = len(class_labels)

# Print class distribution in the train, valid, and test sets
print("Train set class distribution:", Counter(train_generator.classes))
print("Valid set class distribution:", Counter(valid_generator.classes))
print("Test set class distribution:", Counter(test_generator.classes))

Found 3839 images belonging to 4 classes.
Found 2242 images belonging to 4 classes.
Found 319 images belonging to 4 classes.
Train set class distribution: Counter({2: 1920, 3: 1344, 0: 537, 1: 38})
Valid set class distribution: Counter({2: 1120, 3: 784, 0: 315, 1: 23})
Test set class distribution: Counter({2: 160, 3: 112, 0: 44, 1: 3})


In [14]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Define the input shape
input_shape = (64, 64, 3)  # Example: 64x64 RGB images

# Create the model
model = Sequential()

# Convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output of the convolutional layers
model.add(Flatten())

# Dense layers
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))  # num_classes: number of output classes

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_5 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten_2 (Flatten)         (None, 12544)             0         
                                                                 
 dense_6 (Dense)             (None, 128)              

In [21]:
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import os

# Define the paths to your data folders
train_path = '/content/data/train'
valid_path = '/content/data/valid'
test_path = '/content/data/test'

# Define the hyperparameters
batch_size = 16
epochs =30
learning_rate = 0.01
momentum = 0.9

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=True)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define the image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the train, valid, and test data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

checkpoint = ModelCheckpoint('Alzhiemer_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', save_freq='epoch')
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint]
)

# Load the best model
model.save('Alzhiemer_model.h5')

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

Found 3839 images belonging to 4 classes.
Found 2242 images belonging to 4 classes.
Found 319 images belonging to 4 classes.
Epoch 1/30
237/239 [============================>.] - ETA: 0s - loss: 0.0760 - accuracy: 0.9773
Epoch 1: val_accuracy improved from -inf to 0.94866, saving model to Alzhiemer_model.h5
239/239 [==============================] - 6s 23ms/step - loss: 0.0758 - accuracy: 0.9775 - val_loss: 0.2199 - val_accuracy: 0.9487
Epoch 2/30
235/239 [============================>.] - ETA: 0s - loss: 0.0725 - accuracy: 0.9800
Epoch 2: val_accuracy improved from 0.94866 to 0.95179, saving model to Alzhiemer_model.h5
239/239 [==============================] - 4s 16ms/step - loss: 0.0717 - accuracy: 0.9801 - val_loss: 0.1967 - val_accuracy: 0.9518
Epoch 3/30
239/239 [==============================] - ETA: 0s - loss: 0.0754 - accuracy: 0.9780
Epoch 3: val_accuracy did not improve from 0.95179
239/239 [==============================] - 4s 16ms/step - loss: 0.0754 - accuracy: 0.9780 - v

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
<ipython-input-21-c5b4349412a8>:65: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  test_loss, test_accuracy = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)


Test loss: 0.3348
Test accuracy: 0.9309


In [22]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Load the saved model
model = load_model('/content/Alzhiemer_model.h5')

# Load the class labels
class_labels = {value: key for key, value in train_generator.class_indices.items()}

# Ask the user to provide the image path
image_path = input("Enter the path to the image: ")

# Load and preprocess the image
img = image.load_img(image_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make the prediction
prediction = model.predict(x)

# Convert the prediction to a class label
predicted_class = class_labels[np.argmax(prediction)]

# Print the predicted label
print(f"Predicted label: {predicted_class}")

Enter the path to the image: /content/drive/MyDrive/data/test/Moderate_Demented/moderate_34.jpg
1/1 [==============================] - 0s 328ms/step
Predicted label: Moderate_Demented


In [23]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Load the saved model
model = load_model('/content/Alzhiemer_model.h5')

# Load the class labels
class_labels = {value: key for key, value in train_generator.class_indices.items()}

# Ask the user to provide the image path
image_path = input("Enter the path to the image: ")

# Load and preprocess the image
img = image.load_img(image_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make the prediction
prediction = model.predict(x)

# Convert the prediction to a class label
predicted_class = class_labels[np.argmax(prediction)]

# Print the predicted label
print(f"Predicted label: {predicted_class}")

Enter the path to the image: /content/drive/MyDrive/data/test/Non_Demented/non_1049.jpg
1/1 [==============================] - 0s 69ms/step
Predicted label: Non_Demented


In [24]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Load the saved model
model = load_model('/content/Alzhiemer_model.h5')

# Load the class labels
class_labels = {value: key for key, value in train_generator.class_indices.items()}

# Ask the user to provide the image path
image_path = input("Enter the path to the image: ")

# Load and preprocess the image
img = image.load_img(image_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make the prediction
prediction = model.predict(x)

# Convert the prediction to a class label
predicted_class = class_labels[np.argmax(prediction)]

# Print the predicted label
print(f"Predicted label: {predicted_class}")

Enter the path to the image: /content/drive/MyDrive/data/test/Very_Mild_Demented/verymild_1015.jpg
1/1 [==============================] - 0s 66ms/step
Predicted label: Very_Mild_Demented


In [25]:
import numpy as np
from keras.models import load_model
from keras.preprocessing import image

# Load the saved model
model = load_model('/content/Alzhiemer_model.h5')

# Load the class labels
class_labels = {value: key for key, value in train_generator.class_indices.items()}

# Ask the user to provide the image path
image_path = input("Enter the path to the image: ")

# Load and preprocess the image
img = image.load_img(image_path, target_size=(64, 64))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x / 255.0

# Make the prediction
prediction = model.predict(x)

# Convert the prediction to a class label
predicted_class = class_labels[np.argmax(prediction)]

# Print the predicted label
print(f"Predicted label: {predicted_class}")

Enter the path to the image: /content/drive/MyDrive/data/test/Mild_Demented/mild_226.jpg
1/1 [==============================] - 0s 73ms/step
Predicted label: Mild_Demented


In [26]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K

# Define the input shape
img_rows, img_cols = 224, 224
input_shape = (img_rows, img_cols, 3)

# Initialize the model
model = Sequential()

# Block 1
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv1', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu', padding='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv1'))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv1'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv2'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv3'))
model.add(Conv2D(256, (3, 3), activation='relu', padding='same', name='block3_conv4'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv3'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block4_conv4'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv1'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv2'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv3'))
model.add(Conv2D(512, (3, 3), activation='relu', padding='same', name='block5_conv4'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))

# Classifier layers
model.add(Flatten(name='flatten'))
model.add(Dense(4096, activation='relu', name='fc1'))
model.add(Dropout(0.5, name='dropout1'))
model.add(Dense(4096, activation='relu', name='fc2'))
model.add(Dropout(0.5, name='dropout2'))
model.add(Dense(num_classes, activation='softmax', name='predictions'))

# Print the model summary
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0         
                                                                 
 block3_conv1 (Conv2D)       (None, 56, 56, 256)      

In [27]:
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
import tensorflow as tf
import os

# Define the paths to your data folders
train_path = '/content/data/train'
valid_path = '/content/data/valid'
test_path = '/content/data/test'
img_width, img_height = 224,224
# Define the hyperparameters
batch_size = 16
epochs =25
learning_rate = 0.01
momentum = 0.9

optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=momentum, nesterov=True)

# Compile the model
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Define the image data generators
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load the train, valid, and test data
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

checkpoint = ModelCheckpoint('vgg_model.h5', monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=True, mode='auto', save_freq='epoch')
# Train the model
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.samples // batch_size,
    epochs=epochs,
    callbacks=[checkpoint]
)

# Load the best model
model.save('vgg_model.h5')

# Evaluate on the test set
test_loss, test_accuracy = model.evaluate_generator(test_generator, steps=test_generator.samples // batch_size)
print(f'Test loss: {test_loss:.4f}')
print(f'Test accuracy: {test_accuracy:.4f}')

Found 3839 images belonging to 4 classes.
Found 2242 images belonging to 4 classes.
Found 319 images belonging to 4 classes.
Epoch 1/25
239/239 [==============================] - ETA: 0s - loss: 1.0671 - accuracy: 0.4915
Epoch 1: val_accuracy improved from -inf to 0.49955, saving model to vgg_model.h5
239/239 [==============================] - 112s 414ms/step - loss: 1.0671 - accuracy: 0.4915 - val_loss: 1.0452 - val_accuracy: 0.4996
Epoch 2/25
239/239 [==============================] - ETA: 0s - loss: 1.0440 - accuracy: 0.4936
Epoch 2: val_accuracy did not improve from 0.49955
239/239 [==============================] - 73s 304ms/step - loss: 1.0440 - accuracy: 0.4936 - val_loss: 1.0549 - val_accuracy: 0.3496
Epoch 3/25
239/239 [==============================] - ETA: 0s - loss: 1.0419 - accuracy: 0.4931
Epoch 3: val_accuracy improved from 0.49955 to 0.50000, saving model to vgg_model.h5
239/239 [==============================] - 80s 333ms/step - loss: 1.0419 - accuracy: 0.4931 - val_lo

KeyboardInterrupt: 